In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1162, -0.1177, -0.1275,  0.0733,  0.0090,  0.0696, -0.1951, -0.0884,
          0.2102,  0.0055],
        [ 0.0358, -0.1404, -0.2101,  0.0355,  0.0269,  0.1294, -0.0405, -0.0154,
          0.2726, -0.1079]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.0544, -0.1330, -0.0202, -0.0783, -0.0420, -0.0523,  0.0188, -0.0294,
          0.1312,  0.0818],
        [ 0.2278, -0.2318, -0.0533, -0.0897, -0.0384,  0.0620, -0.0904, -0.2274,
          0.0748,  0.1214]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0350, -0.1792, -0.0774,  0.2664,  0.0718,  0.2090, -0.1336, -0.1176,
         -0.0780, -0.0359],
        [ 0.0070, -0.2424,  0.0106,  0.1347,  0.0250,  0.0840, -0.1131, -0.0338,
         -0.0345, -0.1109]], grad_fn=<AddmmBackward0>)

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()  # 在这里矩阵变成一个数了

In [10]:
net = FixedHiddenMLP()
net(X)

tensor([[-0.5775,  0.0519, -0.1466,  0.0081,  0.2577,  0.1087,  0.1064, -0.5436,
          0.2775,  0.0053,  0.6572, -0.3891,  0.0887, -0.3130, -0.0304,  0.3155,
         -0.2035,  0.0350, -0.2417, -0.0166],
        [-0.4716,  0.0356, -0.5033,  0.3169,  0.6464, -0.0155,  0.1868, -0.2229,
          0.3574, -0.0353,  0.5042, -0.5126,  0.1310, -0.1808, -0.3099,  0.0902,
         -0.1355, -0.0114, -0.1185, -0.2824]], grad_fn=<AddmmBackward0>)


tensor(-0.0676, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0241, grad_fn=<SumBackward0>)